In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler, RobustScaler, LabelEncoder
from keras.models import Sequential, load_model
from keras.layers import Input, Dense, Activation, Dropout
from keras.regularizers import l1_l2
from keras.callbacks import ModelCheckpoint

### 1. TensorFlow:
Взять существующий код для простой линейной регрессии (из блокнота Tensorflow examples 1) и переделать его в модель для решения задачи классификации. 

### 1.1 Random dataset

In [3]:
from sklearn.datasets import make_classification

# Step 1: init dataset
X_data, y_data = make_classification(n_samples=2000, n_classes=2, n_features=5,
                                     n_informative=3, n_redundant=2, random_state=42)

# One-hot encoding for labels
y_data = np.array([y_data, -(y_data-1)]).T

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42) 
n_train_samples = X_train.shape[0]
n_test_samples = X_test.shape[0]

In [4]:
rand_clf_X_train, rand_clf_X_test, rand_clf_y_train, rand_clf_y_test = X_train, X_test, y_train, y_test

In [5]:
N_EPOCHS = 20
BATCH_SIZE = 20
LEARNING_RATE = 0.01
n_classes = 2
n_features = 5

In [6]:
# Step 2: create placeholders for X and Y
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
Y = tf.placeholder(tf.float32, shape=(None, n_classes), name='Y')

# Step 3: create weight and bias, initialized to 0
W = tf.Variable(tf.zeros([n_features, n_classes]))
b = tf.Variable(tf.zeros([n_classes]))

# Step 4: build model to predict Y
Y_predicted = tf.nn.sigmoid(tf.matmul(X, W) + b)
# or
# logits = tf.matmul(X, W) + b

# Step 5: use cross entropy as the loss function
loss = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(Y_predicted), reduction_indices=1))
# or
# entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels='Y')
# loss = tf.reduce_mean(entropy)


# Step 6: using gradient descent with learning rate of 0.001 to minimize loss
optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)

# Step 7: calculate accuracy with test set
correct_preds = tf.equal(tf.argmax(Y_predicted, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

with tf.Session() as sess:
    # Step 8: initialize the necessary variables, in this case, w and b
    sess.run(tf.global_variables_initializer()) 
    
    # Step 9: train the model
    for i in range(1, N_EPOCHS+1): 
        total_loss = 0
        n_batches = int(n_train_samples/BATCH_SIZE)
        for j in range(n_batches):
            indexes = np.random.choice(n_train_samples, BATCH_SIZE)
            X_batch, Y_batch = X_train[indexes], y_train[indexes]
            _, loss_batch = sess.run([optimizer, loss], {X: X_batch, Y:Y_batch}) 
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    
    # Step 10: test the model
    print("Accuracy:", accuracy.eval({X: X_test, Y: y_test}))

Average loss epoch 1: 0.5199011374264956
Average loss epoch 2: 0.33693688865751026
Average loss epoch 3: 0.25311205461621283
Average loss epoch 4: 0.204761229082942
Average loss epoch 5: 0.17793664168566464
Average loss epoch 6: 0.15619511669501662
Average loss epoch 7: 0.1371015662327409
Average loss epoch 8: 0.1364249404054135
Average loss epoch 9: 0.11322052618488669
Average loss epoch 10: 0.11729976823553442
Average loss epoch 11: 0.10393875571899117
Average loss epoch 12: 0.10380841335281729
Average loss epoch 13: 0.08560977661982179
Average loss epoch 14: 0.0909971239976585
Average loss epoch 15: 0.08282982306554913
Average loss epoch 16: 0.08140875990502536
Average loss epoch 17: 0.07453312771394849
Average loss epoch 18: 0.08013663971796632
Average loss epoch 19: 0.06996472114697098
Average loss epoch 20: 0.0691787117626518
Accuracy: 0.8225


Чтобы не копипастить этот код для работы с остальными датасетами напишем функцию для обучения логистической регрессии, в которой модель будет обучаться и сохраняться в указанное место:

In [7]:
def train_log_reg(X_train, y_train, epochs, batch_size, learning_rate, n_classes, path_model):
    n_features = X_train.shape[1]
    n_train_samples = X_train.shape[0]
    
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
    Y = tf.placeholder(tf.float32, shape=(None, n_classes), name='Y')

    W = tf.Variable(tf.truncated_normal((n_features, n_classes), name='weights'))
    b = tf.Variable(tf.zeros((n_classes)), name='bias')
    
    logits = tf.matmul(X, W) + b
    
    if n_classes > 2:
        entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y, name='loss')
    else:
        entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
        
    loss = tf.reduce_mean(entropy)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    preds = tf.nn.softmax(logits) if n_classes > 2 else tf.nn.sigmoid(logits)
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))
    confusion_matrix = tf.confusion_matrix(labels=tf.argmax(Y, 1), predictions=tf.argmax(preds, 1),
                                           num_classes=n_classes)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer()) 
        for i in range(1, epochs+1): 
            total_loss = 0
            n_batches = int(n_train_samples/batch_size)
            for j in range(n_batches):
                indexes = np.random.choice(n_train_samples, batch_size)
                X_batch, Y_batch = X_train[indexes], y_train[indexes]
                _, loss_batch = sess.run([optimizer, loss], {X: X_batch, Y:Y_batch}) 
                total_loss += loss_batch
            print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        saver = tf.train.Saver()
        saver.save(sess, path_model)
        print('Model was saved.')
    return X, Y, accuracy, confusion_matrix

In [8]:
def predict_test(X_test, y_test, path_model, clf_report=False):
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, path_model)
        print('Accuracy for test : {}'.format(accuracy.eval({X: X_test, Y: y_test})))
        if clf_report:
            conf = confusion_matrix.eval({X: X_test, Y: y_test})
            TP = np.diag(conf)
            FP = np.sum(conf, axis=0) - TP
            FN = np.sum(conf, axis=1) - TP
            num_classes = conf.shape[0]
            TN = []
            for i in range(num_classes):
                temp = np.delete(conf, i, 0)    # delete ith row
                temp = np.delete(temp, i, 1)  # delete ith column
                TN.append(sum(sum(temp)))
            precision = TP / (TP + FP)
            recall = TP / (TP + FN)
            f1 = 2 * precision * recall / (precision + recall)
            return pd.DataFrame([precision, recall, f1], columns=['Precision', 'Recall', 'F1'], index=range(1, num_classes+1))

In [9]:
X, Y, accuracy, confusion_matrix = train_log_reg(X_train, y_train, 
                                                 epochs=20, batch_size=20,
                                                 learning_rate=0.01, n_classes=2,
                                                 path_model='./models/simple_clf.ckpt')

Average loss epoch 1: 0.9183258205652237
Average loss epoch 2: 0.6139610275626183
Average loss epoch 3: 0.4838238596916199
Average loss epoch 4: 0.4190639529377222
Average loss epoch 5: 0.3903151357546449
Average loss epoch 6: 0.3544321464374661
Average loss epoch 7: 0.34322663079947235
Average loss epoch 8: 0.3467656448483467
Average loss epoch 9: 0.3091584509238601
Average loss epoch 10: 0.309259288944304
Average loss epoch 11: 0.3163765277713537
Average loss epoch 12: 0.2939495490863919
Average loss epoch 13: 0.30257267840206625
Average loss epoch 14: 0.2923426557332277
Average loss epoch 15: 0.3009375812485814
Average loss epoch 16: 0.3027746107429266
Average loss epoch 17: 0.2747647317126393
Average loss epoch 18: 0.2760469681583345
Average loss epoch 19: 0.28648398350924253
Average loss epoch 20: 0.26035088673233986
Model was saved.


In [10]:
predict_test(X_test, y_test, './models/simple_clf.ckpt')

INFO:tensorflow:Restoring parameters from ./models/simple_clf.ckpt
Accuracy for test : 0.875


### 1.2 Titanic

In [23]:
def preprocess_titanic(X):
    scaler = MaxAbsScaler()
    X = X.drop(['Name', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)
    X = X.dropna()
    X['Sex'] = X['Sex'].map({'male': 1, 'female': 0})
    X['Age'] = scaler.fit_transform(X['Age'].values.reshape(-1,1))
    labels = X['Survived']
    return X.drop('Survived', axis=1), labels

In [35]:
data = pd.read_csv('./data/titanic.csv', index_col=0, sep='\t')
data, labels = preprocess_titanic(data)
labels_onehot = pd.get_dummies(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels_onehot, stratify=labels,
                                                    test_size=0.2, random_state=42)

In [36]:
titanic_X_train, titanic_X_test, titanic_y_train, titanic_y_test = X_train, X_test, y_train, y_test

In [38]:
X, Y, accuracy, confusion_matrix = train_log_reg(X_train.values, y_train.values, 
                               epochs=100, batch_size=20,
                               learning_rate=0.1, n_classes=2, path_model='./models/titanic.ckpt')

Average loss epoch 1: 0.7540777564048767
Average loss epoch 2: 0.8711386799812317
Average loss epoch 3: 0.8062511086463928
Average loss epoch 4: 0.7965465068817139
Average loss epoch 5: 0.6868812084197998
Average loss epoch 6: 0.6877895951271057
Average loss epoch 7: 0.6744686126708984
Average loss epoch 8: 0.7392046570777893
Average loss epoch 9: 0.7700897336006165
Average loss epoch 10: 0.6694975614547729
Average loss epoch 11: 0.6656660795211792
Average loss epoch 12: 0.6380314588546753
Average loss epoch 13: 0.6495214462280273
Average loss epoch 14: 0.7071293950080871
Average loss epoch 15: 0.6317631125450134
Average loss epoch 16: 0.6714646697044373
Average loss epoch 17: 0.6456583738327026
Average loss epoch 18: 0.5868741929531097
Average loss epoch 19: 0.5635051548480987
Average loss epoch 20: 0.5814467549324036
Average loss epoch 21: 0.6441594004631043
Average loss epoch 22: 0.6641549348831177
Average loss epoch 23: 0.6003869056701661
Average loss epoch 24: 0.5483025908470154
A

In [39]:
predict_test(X_test, y_test, './models/titanic.ckpt')

INFO:tensorflow:Restoring parameters from ./models/titanic.ckpt
Accuracy for test : 0.8846153616905212


### 1.3 Thyroid_Disease

In [53]:
df = pd.read_csv('./data/preprocessed_Thyroid_Disease.csv', index_col=0, na_values='?')
df = df[df['Class'] != 'secondary_hypothyroid']
df.head()

,age,sex,on_thyroxine,query_on_thyroxine,sick,pregnant,lithium,goitre,hypopituitary,TSH,T3,TT4,T4U,FTI,ref_SVHC,Class
0,-0.419355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.048638,0.652174,0.611106,0.850000,0.064516,1.0,negative
1,-1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.211204,0.000000,-0.027783,0.118183,-0.064208,0.0,negative
2,-0.258065,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.192620,-0.390416,0.166661,-0.300000,0.419355,0.0,negative
3,0.516129,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.561574,-0.130435,1.999994,0.317606,2.038071,0.0,negative
4,0.516129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.309606,-1.043478,-1.166672,-0.500000,-1.193548,0.0,negative


In [54]:
labels = df['Class']
labels_onehot = pd.get_dummies(labels)
df.drop('Class', axis=1, inplace=True)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(df, labels_onehot, stratify=labels,
                                                    test_size=0.25, random_state=42)

In [56]:
TD_X_train, TD_X_test, TD_y_train, TD_y_test = X_train, X_test, y_train, y_test

In [57]:
X, Y, accuracy, confusion_matrix = train_log_reg(X_train.values, y_train.values, 
                               epochs=20, batch_size=20,
                               learning_rate=0.01, n_classes=3, path_model='./models/TD.ckpt')

Average loss epoch 1: 1.0582122210915208
Average loss epoch 2: 0.5835154990777902
Average loss epoch 3: 0.4278638374509541
Average loss epoch 4: 0.35535573790259395
Average loss epoch 5: 0.3190276506522023
Average loss epoch 6: 0.27588453302358057
Average loss epoch 7: 0.2743888726154118
Average loss epoch 8: 0.2530796428521474
Average loss epoch 9: 0.24477253965240844
Average loss epoch 10: 0.212296049672027
Average loss epoch 11: 0.2357821928218324
Average loss epoch 12: 0.24287026666157635
Average loss epoch 13: 0.22537983134917333
Average loss epoch 14: 0.2722994661658791
Average loss epoch 15: 0.23018625597898842
Average loss epoch 16: 0.21168142174046936
Average loss epoch 17: 0.21367657398606868
Average loss epoch 18: 0.21825344287228923
Average loss epoch 19: 0.19981994855065718
Average loss epoch 20: 0.23675515367946726
Model was saved.


In [58]:
predict_test(X_test, y_test, './models/TD.ckpt', clf_report=True)

INFO:tensorflow:Restoring parameters from ./models/TD.ckpt
Accuracy for test : 0.9363732933998108


/home/data_analyst/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


,Precision,Recall,F1
1,NaN,0.948352,0.606061
2,0.0,0.990815,0.833333
3,NaN,0.969118,0.701754


### 2. Keras:
Проделаем тоже самое в Keras

### 2.1 Random dataset

In [59]:
log_reg = Sequential()
log_reg.add(Dense(2, input_dim=5, activation='sigmoid'))
log_reg.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

hist = log_reg.fit(rand_clf_X_train, rand_clf_y_train, batch_size=20, epochs=20, verbose=1,
            validation_data=(rand_clf_X_test, rand_clf_y_test))

Train on 1600 samples, validate on 400 samples
Epoch 1/20
1600/1600 [==============================] - 1s 410us/step - loss: 0.5631 - acc: 0.6644 - val_loss: 0.4906 - val_acc: 0.7438
Epoch 2/20
1600/1600 [==============================] - 0s 307us/step - loss: 0.4137 - acc: 0.8100 - val_loss: 0.4136 - val_acc: 0.8212
Epoch 3/20
1600/1600 [==============================] - 0s 170us/step - loss: 0.3564 - acc: 0.8678 - val_loss: 0.3805 - val_acc: 0.8387
Epoch 4/20
1600/1600 [==============================] - 0s 151us/step - loss: 0.3294 - acc: 0.8822 - val_loss: 0.3625 - val_acc: 0.8400
Epoch 5/20
1600/1600 [==============================] - 0s 147us/step - loss: 0.3137 - acc: 0.8847 - val_loss: 0.3511 - val_acc: 0.8412
Epoch 6/20
1600/1600 [==============================] - 0s 150us/step - loss: 0.3033 - acc: 0.8897 - val_loss: 0.3432 - val_acc: 0.8512
Epoch 7/20
1600/1600 [==============================] - 0s 159us/step - loss: 0.2958 - acc: 0.8928 - val_loss: 0.3370 - val_acc: 0.8550
E

In [60]:
print('Accuracy for test: {}'.format(hist.history['val_acc'][-1]))

Accuracy for test: 0.8824999958276749


### 2.2 Titanic

In [63]:
log_reg = Sequential()
log_reg.add(Dense(2, input_dim=4, activation='sigmoid'))
log_reg.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

hist = log_reg.fit(titanic_X_train, titanic_y_train, batch_size=20, epochs=100, verbose=1,
            validation_data=(titanic_X_test, titanic_y_test))

Train on 100 samples, validate on 26 samples
Epoch 1/100
100/100 [==============================] - 0s 1ms/step - loss: 0.7800 - acc: 0.6150 - val_loss: 0.6510 - val_acc: 0.7308
Epoch 2/100
100/100 [==============================] - 0s 142us/step - loss: 0.7764 - acc: 0.6150 - val_loss: 0.6489 - val_acc: 0.7308
Epoch 3/100
100/100 [==============================] - 0s 142us/step - loss: 0.7730 - acc: 0.6150 - val_loss: 0.6469 - val_acc: 0.7308
Epoch 4/100
100/100 [==============================] - 0s 146us/step - loss: 0.7696 - acc: 0.6200 - val_loss: 0.6449 - val_acc: 0.7115
Epoch 5/100
100/100 [==============================] - 0s 174us/step - loss: 0.7663 - acc: 0.6200 - val_loss: 0.6430 - val_acc: 0.7115
Epoch 6/100
100/100 [==============================] - 0s 183us/step - loss: 0.7634 - acc: 0.6200 - val_loss: 0.6412 - val_acc: 0.7115
Epoch 7/100
100/100 [==============================] - 0s 188us/step - loss: 0.7607 - acc: 0.6200 - val_loss: 0.6394 - val_acc: 0.7115
Epoch 8/100


100/100 [==============================] - 0s 213us/step - loss: 0.6681 - acc: 0.6200 - val_loss: 0.5867 - val_acc: 0.6923
Epoch 61/100
100/100 [==============================] - 0s 175us/step - loss: 0.6670 - acc: 0.6200 - val_loss: 0.5861 - val_acc: 0.6923
Epoch 62/100
100/100 [==============================] - 0s 160us/step - loss: 0.6658 - acc: 0.6200 - val_loss: 0.5855 - val_acc: 0.6923
Epoch 63/100
100/100 [==============================] - 0s 159us/step - loss: 0.6648 - acc: 0.6200 - val_loss: 0.5849 - val_acc: 0.7115
Epoch 64/100
100/100 [==============================] - 0s 157us/step - loss: 0.6637 - acc: 0.6200 - val_loss: 0.5843 - val_acc: 0.7115
Epoch 65/100
100/100 [==============================] - 0s 182us/step - loss: 0.6626 - acc: 0.6200 - val_loss: 0.5837 - val_acc: 0.7115
Epoch 66/100
100/100 [==============================] - 0s 191us/step - loss: 0.6616 - acc: 0.6200 - val_loss: 0.5831 - val_acc: 0.7115
Epoch 67/100
100/100 [==============================] - 0s 17

In [64]:
print('Accuracy for test: {}'.format(hist.history['val_acc'][-1]))

Accuracy for test: 0.749999986245082


### 2.3 Thyroid_Disease

In [65]:
def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

In [66]:
@as_keras_metric
def auc_pr(y_true, y_pred, curve='PR'):
    return tf.metrics.auc(y_true, y_pred, curve=curve, summation_method='careful_interpolation')

In [67]:
log_reg = Sequential()
log_reg.add(Dense(3, input_dim=15, activation='softmax'))
log_reg.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=[auc_pr])

hist = log_reg.fit(TD_X_train, TD_y_train, batch_size=20, epochs=20, verbose=1,
            validation_data=(TD_X_test, TD_y_test))

Train on 2827 samples, validate on 943 samples
Epoch 1/20
2827/2827 [==============================] - 1s 357us/step - loss: 0.8727 - auc_pr: 0.4739 - val_loss: 0.6228 - val_auc_pr: 0.6845
Epoch 2/20
2827/2827 [==============================] - 1s 337us/step - loss: 0.5024 - auc_pr: 0.7780 - val_loss: 0.4224 - val_auc_pr: 0.8405
Epoch 3/20
2827/2827 [==============================] - 1s 219us/step - loss: 0.3725 - auc_pr: 0.8728 - val_loss: 0.3543 - val_auc_pr: 0.8946
Epoch 4/20
2827/2827 [==============================] - 1s 218us/step - loss: 0.3140 - auc_pr: 0.9076 - val_loss: 0.3147 - val_auc_pr: 0.9182
Epoch 5/20
2827/2827 [==============================] - 1s 358us/step - loss: 0.2852 - auc_pr: 0.9262 - val_loss: 0.2834 - val_auc_pr: 0.9316
Epoch 6/20
2827/2827 [==============================] - 1s 265us/step - loss: 0.2656 - auc_pr: 0.9362 - val_loss: 0.2761 - val_auc_pr: 0.9370
Epoch 7/20
2827/2827 [==============================] - 1s 227us/step - loss: 0.2571 - auc_pr: 0.9430

In [68]:
print('PR_auc for test: {}'.format(hist.history['val_auc_pr'][-1]))

PR_auc for test: 0.9717485174015243


### 2.4 Boston dataset:

In [69]:
from sklearn.datasets import load_boston
scaler = RobustScaler()
boston = load_boston()
X = scaler.fit_transform(boston.data)
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [72]:
regression = Sequential()
# regression.add(Dense(13, input_dim=13, activation='relu'))
regression.add(Dense(1, input_dim=13))
regression.compile(loss='mean_squared_error', optimizer='sgd')

hist = regression.fit(X_train, y_train, batch_size=10, epochs=20, verbose=1,
                      validation_data=(X_test, y_test))

Train on 379 samples, validate on 127 samples
Epoch 1/20
379/379 [==============================] - 0s 1ms/step - loss: 267.1233 - val_loss: 126.3030
Epoch 2/20
379/379 [==============================] - 0s 521us/step - loss: 104.8280 - val_loss: 67.2553
Epoch 3/20
379/379 [==============================] - 0s 337us/step - loss: 59.1381 - val_loss: 46.8592
Epoch 4/20
379/379 [==============================] - 0s 327us/step - loss: 43.4062 - val_loss: 34.2825
Epoch 5/20
379/379 [==============================] - 0s 301us/step - loss: 34.9173 - val_loss: 33.2025
Epoch 6/20
379/379 [==============================] - 0s 304us/step - loss: 30.6958 - val_loss: 35.1250
Epoch 7/20
379/379 [==============================] - 0s 281us/step - loss: 30.4011 - val_loss: 27.9073
Epoch 8/20
379/379 [==============================] - 0s 279us/step - loss: 29.4844 - val_loss: 30.4638
Epoch 9/20
379/379 [==============================] - 0s 280us/step - loss: 27.7331 - val_loss: 27.8862
Epoch 10/20
379/3

In [73]:
print('MSE for test (1 neuron): {}'.format(hist.history['val_loss'][-1]))

MSE for test (1 neuron): 28.301932676570623


In [355]:
print('MSE for test (with hidden layer): {}'.format(hist.history['val_loss'][-1]))

MSE for test (with hidden layer): 18.329604409811065


### 2.5 Titanic with multilayer perceptron

Baseline который был выше:

In [356]:
log_reg = Sequential()
log_reg.add(Dense(2, input_dim=4, activation='sigmoid'))
log_reg.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

hist = log_reg.fit(titanic_X_train, titanic_y_train, batch_size=20, epochs=100, verbose=1,
            validation_data=(titanic_X_test, titanic_y_test))

Train on 571 samples, validate on 143 samples
Epoch 1/100
571/571 [==============================] - 0s 683us/step - loss: 1.3113 - acc: 0.4247 - val_loss: 1.1843 - val_acc: 0.4301
Epoch 2/100
571/571 [==============================] - 0s 107us/step - loss: 1.1136 - acc: 0.4536 - val_loss: 1.0140 - val_acc: 0.4441
Epoch 3/100
571/571 [==============================] - 0s 101us/step - loss: 0.9647 - acc: 0.5114 - val_loss: 0.8892 - val_acc: 0.5105
Epoch 4/100
571/571 [==============================] - 0s 89us/step - loss: 0.8590 - acc: 0.5236 - val_loss: 0.8045 - val_acc: 0.5000
Epoch 5/100
571/571 [==============================] - 0s 93us/step - loss: 0.7889 - acc: 0.5166 - val_loss: 0.7505 - val_acc: 0.5000
Epoch 6/100
571/571 [==============================] - 0s 93us/step - loss: 0.7448 - acc: 0.4965 - val_loss: 0.7167 - val_acc: 0.5070
Epoch 7/100
571/571 [==============================] - 0s 98us/step - loss: 0.7167 - acc: 0.4860 - val_loss: 0.6950 - val_acc: 0.5105
Epoch 8/100
5

In [357]:
print('Accuracy for test: {}'.format(hist.history['val_acc'][-1]))

Accuracy for test: 0.7797202776362012


Добавим еще скрытый слой и увеличим чило нейронов на входном слое. Так же во время обучения и валидации сохраним ту модель, которая покажет наилучшую accuracy на валидационном датасете.

In [392]:
log_reg = Sequential()
log_reg.add(Dense(8, input_dim=4, kernel_initializer='glorot_normal', activation='relu'))
log_reg.add(Dense(20, kernel_initializer='glorot_normal', activation='relu'))
log_reg.add(Dense(2, activation='sigmoid'))
log_reg.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# checkpoint
filepath='./models/best_titanic_model.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

hist = log_reg.fit(titanic_X_train, titanic_y_train, batch_size=3, epochs=20, verbose=1,
                   validation_data=(titanic_X_test, titanic_y_test), callbacks=callbacks_list)

Train on 571 samples, validate on 143 samples
Epoch 1/20
571/571 [==============================] - 1s 2ms/step - loss: 0.6500 - acc: 0.6095 - val_loss: 0.6290 - val_acc: 0.6049

Epoch 00001: val_acc improved from -inf to 0.60490, saving model to ./models/best_titanic_model.hdf5
Epoch 2/20
571/571 [==============================] - 0s 534us/step - loss: 0.6166 - acc: 0.6235 - val_loss: 0.5974 - val_acc: 0.6713

Epoch 00002: val_acc improved from 0.60490 to 0.67133, saving model to ./models/best_titanic_model.hdf5
Epoch 3/20
571/571 [==============================] - 0s 547us/step - loss: 0.5818 - acc: 0.7049 - val_loss: 0.5542 - val_acc: 0.7587

Epoch 00003: val_acc improved from 0.67133 to 0.75874, saving model to ./models/best_titanic_model.hdf5
Epoch 4/20
571/571 [==============================] - 0s 560us/step - loss: 0.5442 - acc: 0.7574 - val_loss: 0.5190 - val_acc: 0.7867

Epoch 00004: val_acc improved from 0.75874 to 0.78671, saving model to ./models/best_titanic_model.hdf5
Epo

In [395]:
model = load_model('./models/best_titanic_model.hdf5')

In [399]:
print('Accuracy for test: {}'.format(model.evaluate(titanic_X_test, titanic_y_test)[1]))

143/143 [==============================] - 0s 77us/step
Accuracy for test: 0.8111888124392583
